In [36]:

import string
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

In [5]:
df = pd.read_csv(r'C:\Users\Sahana\Aggression-Detection\dataset\raw_data.csv')
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
#drop rows with NaN value
df.dropna(subset = ["tweet"], inplace=True)
df.drop_duplicates()
df.shape
df.head()

,Unnamed: 0,tweet,class
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [2]:
#basic pre-processing using tweet-preprocessor
'''
removes :
URLs
Hashtags
Mentions
Reserved words (RT, FAV)
Emojis
Smileys
'''
!pip  install "C:\Users\Sahana\tweet-preprocessor-0.4.0"



Processing c:\users\sahana\tweet-preprocessor-0.4.0
  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.4.0-py3-none-any.whl size=7573 sha256=8a5c47d110920bc3ec6d521eb55ed540ab80fb80bea841b5c5ff66fc586ae73e
  Stored in directory: c:\users\sahana\appdata\local\pip\cache\wheels\a9\34\79\bb0aefe11348ea0bc31bffb8186b9e13ef58788379f51b9528
Successfully built tweet-preprocessor


In [17]:
import preprocessor as p
def tweet_pro(row):
    tweet = row['tweet']
    tweet = p.clean(tweet)
    tweet = tweet.lower()
    tweet = tweet.translate(str.maketrans("","", string.punctuation))
    return tweet

df['tweet'] = df.apply(tweet_pro,axis = 1)

In [27]:

df.to_csv("raw_data1.csv")


In [3]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install -U spacy-lookups-data


  Using cached https://files.pythonhosted.org/packages/0a/19/2b2c0e1340131a8e23ce4a9804cdccdd62d4d23d3d86c1754857b3de7a14/spacy-2.2.4-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/21/e1/e4e7b754e6be3a79c400eb766fb34924a6d278c43bb828f94233e0124a21/wasabi-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7b/98/00e345edf2ef6d66a8c3cd08779a9829f13c76b37bf3c2445e9965881c2f/blis-0.4.1-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/15/8f/0dad3ca706e31258cf7b9adf40f8d2103444a09dd7d66d46cf6980025c65/murmurhash-1.0.2-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/86/85/40b8f66c2dd8f4fd9f09d59b22720cffecf1331e788b8a0cab5bafb353d1/plac-1.1.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/71/a58322c3489bf0f5a71aa69a66b42164cbc4f0d5ac5e1042c11233766b3f/preshed-3.0.2-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packag

  Using cached https://files.pythonhosted.org/packages/3c/f1/be61b032e02a06a221e14f906dc251de90ac459dc2739f0c5225844ecb08/spacy_lookups_data-0.2.0.tar.gz
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.2.0-py2.py3-none-any.whl size=29164788 sha256=2ba26fd3cc29b888fdcd19461a69e7ed7be65e1c3624c522e38f7bc990c4c577
  Stored in directory: C:\Users\Sahana\AppData\Local\pip\Cache\wheels\79\a4\b8\6085d282396938b29675292697e72871b145990d0079ceadc1
Successfully built spacy-lookups-data


In [25]:
#delete empty rows (after pre-processing)

df = pd.read_csv(r'C:\Users\Sahana\Desktop\final year project\final pro\raw_data1.csv')
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
#drop rows with NaN value
df.dropna(axis = 0, subset = ['tweet'], inplace=True)


In [26]:
#tokenization,lemmatization using spaCy

import spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')



def lemmatize(doc):
    doc = doc['tweet']
    doc = nlp(doc)
    tokens = [tokens for tokens in doc if ((tokens.is_stop == False) and tokens.text != '')]
    lemma = [token.lemma_ for token in tokens]
    
    return " ".join(lemma)

df['lemma_text'] = df.apply(lemmatize,axis = 1)


In [57]:
from sklearn import model_selection
train_x, test_x, train_y, test_y = model_selection.train_test_split(df['lemma_text'], df['class'],test_size=0.33, random_state=42)
print(train_x.shape,test_x.shape)

In [1]:
%run ../paths.py
CONST = CONST()

In [58]:
train_x = pd.read_csv(CONST.CLEANED_TRAIN).tweet
test_x = pd.read_csv(CONST.CLEANED_TEST).tweet
train_y = pd.read_csv(CONST.CLEANED_TRAIN)["class"]
test_y = pd.read_csv(CONST.CLEANED_TEST)["class"]

In [59]:
#feature vectors using tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion


#word unigrams,bigrams,trigramsma
word_vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features=10000).fit(train_x)



#feature vectors
x = word_vectorizer.transform(train_x)

#testing feature vectors
x_test = word_vectorizer.transform(test_x)


In [28]:
x.shape

(42210, 10000)

In [60]:
#feature vectors using tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion


#word unigrams,bigrams,trigrams
word_vectorizer = TfidfVectorizer(analyzer = 'word',token_pattern = r'\w+',ngram_range=(1,3)).fit(train_x)

#character n-grams
char_vectorizer =  TfidfVectorizer(analyzer = 'char',token_pattern = r'\w+',ngram_range=(2,8)).fit(train_x)

#combine
vectorizer = FeatureUnion([
('word_vectorizer', TfidfVectorizer(analyzer = 'word',token_pattern = r'\w+',ngram_range=(1,3))),
('char_vectorizer', TfidfVectorizer(analyzer = 'char',token_pattern = r'\w+',ngram_range=(2,8)))]).fit(train_x)


#training feature vectors
x = vectorizer.transform(train_x)
word_x = word_vectorizer.transform(train_x)
char_x = char_vectorizer.transform(train_x)

#testing feature vectors
x_test = vectorizer.transform(test_x)
word_x_test = word_vectorizer.transform(test_x)
char_x_test = char_vectorizer.transform(test_x)



In [61]:
#no. of features/dimensions
print(len(word_vectorizer.vocabulary_))
print(len(char_vectorizer.vocabulary_))

307267
1692965


In [49]:
#delete rows if lemma_text is empty

nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
#drop rows with NaN value
df.dropna(axis = 0, subset = ['lemma_text'], inplace=True)

In [62]:
#an example of a feature vector
print(x[23])

  (0, 56484)	0.17169391165258482
  (0, 56499)	0.2559170011632014
  (0, 56500)	0.2559170011632014
  (0, 61434)	0.17966464498520657
  (0, 61451)	0.2559170011632014
  (0, 61452)	0.2559170011632014
  (0, 90427)	0.2298918358275759
  (0, 114515)	0.2559170011632014
  (0, 114516)	0.2559170011632014
  (0, 114517)	0.2559170011632014
  (0, 136372)	0.2342108742260666
  (0, 136373)	0.2559170011632014
  (0, 136374)	0.2559170011632014
  (0, 290170)	0.22028672164381308
  (0, 290173)	0.2298918358275759
  (0, 303492)	0.09344358735132607
  (0, 303906)	0.2559170011632014
  (0, 303907)	0.2559170011632014
  (0, 338397)	0.033794824008096185
  (0, 342031)	0.03350183016004224
  (0, 342310)	0.04500450194505963
  (0, 342356)	0.050611978222283915
  (0, 342361)	0.05137020168108364
  (0, 342362)	0.05147093528554256
  (0, 342369)	0.07671959522941506
  :	:
  (0, 1845103)	0.06529001088239018
  (0, 1845104)	0.06529001088239018
  (0, 1845108)	0.06891769015373583
  (0, 1845109)	0.06891769015373583
  (0, 1911771)	0.043535

In [33]:
#svm
from sklearn import svm
#word n-grams 
word_clf = svm.SVC(gamma ='auto').fit(word_x,train_y)
word_clf_pre = svm.SVC(gamma = 'auto', kernel = 'linear').fit(word_x,train_y)
word_clf_poly = svm.SVC(gamma = 'auto', kernel = 'poly').fit(word_x,train_y)

#char n-grams
char_clf = svm.SVC(gamma ='auto').fit(char_x,train_y)
char_clf_pre = svm.SVC(gamma = 'auto', kernel = 'linear').fit(char_x,train_y)
char_clf_poly = svm.SVC(gamma = 'auto', kernel = 'poly').fit(char_x,train_y)


#combined fea.
clf = svm.SVC(gamma ='auto').fit(x,train_y)
clf_pre = svm.SVC(gamma = 'auto', kernel = 'linear').fit(x,train_y)
clf_poly = svm.SVC(gamma = 'auto', kernel = 'poly').fit(x,train_y)





In [60]:
#svm
from sklearn import svm

#word n-grams 
word_clf = svm.SVC(gamma ='auto').fit(x,train_y)
word_clf_pre = svm.SVC(gamma = 'auto', kernel = 'linear').fit(x,train_y)
word_clf_poly = svm.SVC(gamma = 'auto', kernel = 'poly').fit(x,train_y)




In [61]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#word n-grams

word_pre = word_clf.predict(x_test)
word_pre_pre= word_clf_pre.predict(x_test)
word_pre_poly= word_clf_poly.predict(x_test)
print('Prediction with word n-gram features .... ')
print("linear kernel ")
print("accuracy -> ",accuracy_score(word_pre_pre, test_y)*100)
print("F1 score ->",f1_score(test_y,word_pre_pre,average='weighted'))
print("rbf kernel")
print("accuracy -> ",accuracy_score(word_pre, test_y)*100)
print("F1 score ->",f1_score(test_y,word_pre,average='weighted'))
print("Polynomial kernel")
print("accuracy -> ",accuracy_score(word_pre_poly, test_y)*100)
print("F1 score ->",f1_score(test_y,word_pre_poly,average='weighted'))

Prediction with word n-gram features .... 
linear kernel 
accuracy ->  94.21494542401344
F1 score -> 0.9417339926458201
rbf kernel
accuracy ->  58.211586901763226
F1 score -> 0.4283616536537966
Polynomial kernel
accuracy ->  58.211586901763226
F1 score -> 0.4283616536537966


In [65]:
#prediction and accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#word n-grams

word_pre = word_clf.predict(word_x_test)
word_pre_pre= word_clf_pre.predict(word_x_test)
word_pre_poly= word_clf_poly.predict(word_x_test)
print('Prediction with word n-gram features .... ')
print("linear kernel ")
print("accuracy -> ",accuracy_score(word_pre_pre, test_y)*100)
print("F1 score ->",f1_score(test_y,word_pre_pre,average='weighted'))
print("rbf kernel")
print("accuracy -> ",accuracy_score(word_pre, test_y)*100)
print("F1 score ->",f1_score(test_y,word_pre,average='weighted'))
print("Polynomial kernel")
print("accuracy -> ",accuracy_score(word_pre_poly, test_y)*100)
print("F1 score ->",f1_score(test_y,word_pre_poly,average='weighted'))



#char n-grams

char_pre = char_clf.predict(char_x_test)
char_pre_pre= char_clf_pre.predict(char_x_test)
char_pre_poly= char_clf_poly.predict(char_x_test)
print('Prediction with char n-gram features .... ')
print("linear kernel ")
print("accuracy -> ",accuracy_score(char_pre_pre, test_y)*100)
print("F1 score ->",f1_score(test_y,char_pre_pre,average='weighted'))
print("rbf kernel")
print("accuracy -> ",accuracy_score(char_pre, test_y)*100)
print("F1 score ->",f1_score(test_y,char_pre,average='weighted'))
print("Polynomial kernel")
print("accuracy -> ",accuracy_score(char_pre_poly, test_y)*100)
print("F1 score ->",f1_score(test_y,char_pre_poly,average='weighted'))



#combined fea. 
pre = clf.predict(x_test)
pre_pre= clf_pre.predict(x_test)
pre_poly= clf_poly.predict(x_test)
print('Prediction with features combined.... ')
print("linear kernel ")
print("accuracy -> ",accuracy_score(pre_pre, test_y)*100)
print("F1 score ->",f1_score(test_y,pre_pre,average='weighted'))
print("rbf kernel")
print("accuracy -> ",accuracy_score(pre, test_y)*100)
print("F1 score ->",f1_score(test_y,pre,average='weighted'))
print("Polynomial kernel")
print("accuracy -> ",accuracy_score(pre_poly, test_y)*100)
print("F1 score ->",f1_score(test_y,pre_poly,average='weighted'))



Prediction with word n-gram features .... 
linear kernel 
accuracy ->  94.13052017282982
F1 score -> 0.9411520366044245
rbf kernel
accuracy ->  58.08316031648055
F1 score -> 0.42682009969891394
Polynomial kernel
accuracy ->  58.08316031648055
F1 score -> 0.42682009969891394


C:\Users\Sahana\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Prediction with char n-gram features .... 
linear kernel 
accuracy ->  94.45597890129622
F1 score -> 0.9443279990037949
rbf kernel
accuracy ->  58.08316031648055
F1 score -> 0.42682009969891394
Polynomial kernel
accuracy ->  58.08316031648055
F1 score -> 0.42682009969891394
Prediction with features combined.... 
linear kernel 
accuracy ->  94.41669939958476
F1 score -> 0.9440887171600834
rbf kernel
accuracy ->  58.08316031648055
F1 score -> 0.42682009969891394
Polynomial kernel
accuracy ->  58.08316031648055
F1 score -> 0.42682009969891394


In [15]:
#Naive bayes 
from sklearn import naive_bayes


nb = naive_bayes.MultinomialNB().fit(x,train_y)
pre = nb.predict(x_test)
print("Naive bayes Accuracy Score -> ",accuracy_score(pre, test_y)*100)
#print("F1 score ->",f1_score(test_y,pre,average='weighted'))

Naive bayes Accuracy Score ->  90.35714285714286
